### Mini-Batch/Batch/Stochastic

In [ ]:
import torch

def default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')   
    if torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

device = default_device()

In [ ]:
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from tqdm import *

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.hideen1 = nn.Linear(1, 32)
        self.hideen2 = nn.Linear(32, 32)
        self.output = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.hideen1(x))
        x = torch.relu(self.hideen2(x))
        return x


In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
np.random.seed(0)
n_samples = 1024
x = np.linspace(-5, 5, n_samples)
y = 0.3 * x * x + np.random.randn(n_samples)

x = torch.unsqueeze(torch.from_numpy(x).float(), 1)
y = torch.unsqueeze(torch.from_numpy(y).float(), 1)

dataset = torch.utils.data.TensorDataset(x, y)

names = ['Batch', "Stochastic", "Mini-batch"]
batch_sizes = [n_samples, 1, 256]
momentum = [1, 0, 1]
losses =[[], [], []]

learning_rate = 0.0001
n_epochs = 1000

for i in range(3):
    model = Model()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum[i])
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_sizes[i], shuffle=True)
    for epoch in tqdm(range(n_epochs), leave=True, desc=names[i], unit=' epoch'):
        x_batch, y_batch = next(iter(loader))
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        losses[i].append(loss.item())


In [ ]:
for i , lost_list in enumerate(losses):
    plt.figure(figsize=(12,4))
    plt.plot(lost_list, label=names[i])
    plt.ylim(0, 15)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(names[i])
    plt.show()
